<h1><center>ERM with DNN under penalty of Equalized Odds</center></h1>

We implement here a regular Empirical Risk Minimization (ERM) of a Deep Neural Network (DNN) penalized to enforce an Equalized Odds constraint. More formally, given a dataset of size $n$ consisting of context features $x$, target $y$ and a sensitive information $z$ to protect, we want to solve
$$
\text{argmin}_{h\in\mathcal{H}}\frac{1}{n}\sum_{i=1}^n \ell(y_i, h(x_i)) + \lambda \chi^2|_1
$$
where $\ell$ is for instance the MSE and the penalty is
$$
\chi^2|_1 = \left\lVert\chi^2\left(\hat{\pi}(h(x)|y, z|y), \hat{\pi}(h(x)|y)\otimes\hat{\pi}(z|y)\right)\right\rVert_1
$$
where $\hat{\pi}$ denotes the empirical density estimated through a Gaussian KDE.

### The dataset

We use here the _communities and crimes_ dataset that can be found on the UCI Machine Learning Repository (http://archive.ics.uci.edu/ml/datasets/communities+and+crime). Non-predictive information, such as city name, state... have been removed and the file is at the arff format for ease of loading.

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))

In [2]:
from examples.data_loading import read_dataset
x_train, y_train, z_train, x_test, y_test, z_test = read_dataset(name='crimes', fold=1)
n, d = x_train.shape

### The Deep Neural Network

We define a very simple DNN for regression here

In [3]:
from torch import nn
import torch.nn.functional as F

class NetRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NetRegression, self).__init__()
        size = 50
        self.first = nn.Linear(input_size, size)
        self.fc = nn.Linear(size, size)
        self.last = nn.Linear(size, num_classes)

    def forward(self, x):
        out = F.selu(self.first(x))
        out = F.selu(self.fc(out))
        out = self.last(out)
        return out

### The fairness-inducing regularizer
We implement now the regularizer. The empirical densities $\hat{\pi}$ are estimated using a Gaussian KDE. The L1 functional norm is taken over the values of $y$.
$$
\chi^2|_1 = \left\lVert\chi^2\left(\hat{\pi}(x|z, y|z), \hat{\pi}(x|z)\otimes\hat{\pi}(y|z)\right)\right\rVert_1
$$
This used to enforce the conditional independence $X \perp Y \,|\, Z$.
Practically, we will want to enforce $\text{prediction} \perp \text{sensitive} \,|\, \text{target}$

In [4]:
from hgr.independence.density_estimation.pytorch_kde import kde
from hgr.independence.hgr import chi_2_cond

def chi_squared_l1_kde(X, Y, Z):
    tmp = chi_2_cond(X, Y, Z, kde)
    # print(tmp.shape, X.shape, Y.shape, Z.shape)
    # print(torch.unique(Z))
    return torch.mean(tmp)

### The fairness-penalized ERM

We now implement the full learning loop. The regression loss used is the quadratic loss with a L2 regularization and the fairness-inducing penalty.

In [8]:
import torch
import numpy as np
import torch.utils.data as data_utils

def regularized_learning(x_train, y_train, z_train, model, fairness_penalty, lr=1e-5, num_epochs=10):
    # wrap dataset in torch tensors
    Y = torch.tensor(y_train.astype(np.float32))
    X = torch.tensor(x_train.astype(np.float32))
    Z = torch.tensor(z_train.astype(np.float32))
    dataset = data_utils.TensorDataset(X, Y, Z)
    dataset_loader = data_utils.DataLoader(dataset=dataset, batch_size=200, shuffle=True)

    # mse regression objective
    data_fitting_loss = nn.MSELoss()

    # stochastic optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.01)

    for j in range(num_epochs):
        for i, (x, y, z) in enumerate(dataset_loader):
            def closure():
                optimizer.zero_grad()
                outputs = model(x).flatten()
                loss = data_fitting_loss(outputs, y)
                loss += fairness_penalty(outputs, z, y)
                loss.backward()
                return loss
            optimizer.step(closure)
        mse_curr_test = evaluate(model, x_test, y_test, z_test)
        print(f"TEST -- mse: {mse_curr_test}")
        mse_curr_train = evaluate(model, x_train, y_train, z_train)
        print(f"TRAIN -- mse: {mse_curr_train}")
    
    return model

### Evaluation

For the evaluation on the test set, we compute two metrics: the MSE (accuracy) and HGR$|_\infty$ (fairness).

In [9]:
from hgr.independence.hgr import hgr_cond

def evaluate(model, x, y, z):
    Y = torch.tensor(y.astype(np.float32))
    Z = torch.Tensor(z.astype(np.float32))
    X = torch.tensor(x.astype(np.float32))

    prediction = model(X).detach().flatten()
    loss = nn.MSELoss()(prediction, Y)
    # hgr_infty = np.max(hgr_cond(prediction, Z, Y, kde))
    return loss.item() #, hgr_infty

### Running everything together


In [11]:
model = NetRegression(d, 1)
num_epochs = 200
lr = 1e-5
penalty = chi_squared_l1_kde

model = regularized_learning(x_train, y_train, z_train, model=model, fairness_penalty=penalty, lr=lr, \
                             num_epochs=num_epochs)

nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.S

nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 10
torch.Size([10, 10, 10, 3])
torch.Size([10, 10, 10])
torch.Size([10, 1, 10]) torch.Size([1, 10, 10])
torch.Size([10, 10, 10])
torch.Size([10])


TEST -- mse: 0.0826299786567688
TRAIN -- mse: 0.09622500836849213
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbin

nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.S

torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 10
torch.Size([10, 10, 10, 3])
torch.Size([10, 10, 10])
torch.Size([10, 1, 10]) torch.Size([1, 10, 10])
torch.Size([10, 10, 10])
torch.Size([10])


TEST -- mse: 0.06856253743171692
TRAIN -- mse: 0.07779295742511749
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
tor

TRAIN -- mse: 0.0724421888589859
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])

...

nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 10
torch.Size([10, 10, 10, 3])
torch.Size([10, 10, 10])
torch.Size([10, 1, 10]) torch.Size([1, 10, 10])
torch.Size([10, 10, 10])
torch.Size([10])


TEST -- mse: 0.03504553437232971
TRAIN -- mse: 0.03614923730492592
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbi

torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 10
torch.Size([10, 10, 10, 3])
torch.Size([10, 10, 10])
torch.Size([10, 1, 10]) torch.Size([1, 10, 10])
torch.Size([10, 10, 10])
torch.Size([10])


TEST -- mse: 0.03382372856140137
TRAIN -- mse: 0.03542306274175644
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
tor

nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 10
torch.Size([10, 10, 10, 3])
torch.Size([10, 10, 10])
torch.Size([10, 1, 10]) torch.Size([1, 10, 10])
torch.Size([10, 10, 10])
torch.Size([10])


TEST -- mse: 0.033688947558403015
TRAIN -- mse: 0.03509102016687393
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nb

nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 10
torch.Size([10, 10, 10, 3])
torch.Size([10, 10, 10])
torch.Size([10, 1, 10]) torch.Size([1, 10, 10])
torch.Size([10, 10, 10])
torch.Size([10])


TEST -- mse: 0.03341338783502579
TRAIN -- mse: 0.03483409062027931
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbi

nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 10
torch.Size([10, 10, 10, 3])
torch.Size([10, 10, 10])
torch.Size([10, 1, 10]) torch.Size([1, 10, 10])
torch.Size([10, 10, 10])
torch.Size([10])


TEST -- mse: 0.03349409997463226
TRAIN -- mse: 0.03461427986621857
nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbins: 11
torch.Size([11, 11, 11, 3])
torch.Size([11, 11, 11])
torch.Size([11, 1, 11]) torch.Size([1, 11, 11])
torch.Size([11, 11, 11])
torch.Size([11])


nbi

In [ ]:
mse, hgr_infty = evaluate(model, x_test, y_test, z_test)
print("MSE:{} HGR_infty:{}".format(mse, hgr_infty))